**2\. Анализ данных (Data Understanding)**

Цель шага – понять слабые и сильные стороны предоставленных данных, определить их достаточность, предложить идеи, как  
их использовать, и лучше понять процессы заказчика. Для этого мы строим графики, делаем выборки и рассчитываем  
статистики.

2.1 Сбор данных (Data collection)  
Выгружаем необходимые данные (или срез данных если их объем слишком велик) из источников.  
Версионируем данные средствами DVC.

2.2 Исследование данных (Data exploration)  
Исследуем данные, чтобы сформулировать гипотезы относительно того, как эти данные помогут решить задачу. Проверяем  
качество данных.

Ориентировочный список для проверки данных:

1. Загрузить репрезентативную выборку из набора данных  
2. Провести предварительный анализ всей выборки.  
* определить тип данных в каждом столбце  
* Категориальные данные (Номинальные, Порядковые)  
* Числовые данные (дискретные, непрерывные, интервальные, отношения)  
* При необходимости преобразовать данные к нужным типам  
* Проверить на выбросы, отсутствующие значения, невалидные значения(например в системе случился сбой и в поле с именемпопала длина просмотра).

(по результатам предварительного анализа сделать визуализацию, обычно это табличка с характеристиками или какой то из  
профайлеров)

3. На основе предыдущего анализа выполнить очистку данных (обработать выбросы, отсутствующие значения, удалить невалидные значения)  
4. Удалить из рассмотрения неинформативные данные. (лишние идентификаторы, служебные поля, поля с очень малым количеством значений)  
5. Провести статистический анализ оставшихся данных  
   1. рассчитать ключевые статистики для каждого типа данных  
   2. построить распределения (тип графика выбрать в зависимости от данных, часто полезно построить гистограмму, но иногда лучше воспользоваться линейным графиком или посмотреть распределение во времени с помощью scaterplot)  
6. Провести корреляционный анализ  
   1. Для количественных данных нормализовать данные и построить матрицу корреляции Пирсона  
   2. сделать выводы на основе матрицы (найти утечки данных, найти важные признаки линейно влияющие на целевой показатель, определить гипотезы по конструированию признаков)  
   3. Для количественных и порядковых данных \- построить матрицу корреляции Спирмена  сделать выводы аналогично предыдущему анализу, только учесть тип данных  
   4. Для всех данных построить матрицу корреляции Пфика  
   5. Сделать выводы на основе анализа, сделать оценку между всеми типами корреляции(на пересекающихся данных),  
   6. попытаться найти объяснения различиям.  
   7. Сделать выводы о наличии или отсутствии нелинейных связей.  
        
7.  Провести обработку данных, на основе выводов полученных в прошлых шагах.  
   1. провести дополнительную очистку  
   2. выполнить нужный тип энкодинга(если требуется)  
   3. Сконструировать новые признаки.

8. Построить Графики взаимодействия полученных данных с целевым показателем  
   1. Для количественных данных линейные графики на нормализованных данных  
   2. для категориальных данных с малым количеством категорий построить ScaterPlot во времени  
   3. для категориальных данных с большим количеством показателей построить heatmap во времени(обычно строят, только для ризнаков которые показывают высокие коэффициенты корреляции и потенциально интересны, пример такого графика спектрограмма в анализе звука)  
9. Сделать выводы на основе проведенного анализа и учитывая особенности планируемой архитектуры модели.  
   1. Какие данные и почему нельзя использовать в модели  
   2. какие данные можно использовать без преобразования  
   3. какие данные можно использовать выполнив преобразование  
   4. какие новые признаки нужно использовать и почему  
   5. есть ли смысл использовать один набор признаков или построить разные модели на подмножестве признаков и почему.  
   6. Выделить итоговый список необходимых данных  
10. Описать ожидания от модели на проанализированных данных

In [ ]:
from lakefs_client import Configuration
from dotenv import load_dotenv
import os

load_dotenv()  # Загружает переменные из .env

def get_lakefs_config():
    config = Configuration()
    config.host = os.getenv("LAKEFS_HOST")
    config.username = os.getenv("LAKEFS_ACCESS_KEY")
    config.password = os.getenv("LAKEFS_SECRET_KEY")
    return config


In [ ]:
from lakefs_client import Configuration
from lakefs_client import ApiClient
from lakefs_client.api import objects_api

def load_dataframe(configuration: Configuration, repository: str, ref: str, path: str):
    # Создаем клиента
    with ApiClient(configuration) as api_client:
        obj_api = objects_api.ObjectsApi(api_client)

        # Получаем объект
        #repository = "mfdp-fin-fraud-detection-data"
        #ref = "main"  # или другой commit/branch/tag
        #path = "ieee-fraud-detection/sample_submission.csv"

        response = obj_api.get_object(repository, ref, path)
        content = response.read()

        # Например, если CSV
        import pandas as pd
        from io import StringIO
        df = pd.read_csv(StringIO(content.decode('utf-8')))
        return df

In [ ]:
# Получаем объект
configuration = get_lakefs_config()
repository = "mfdp-fin-fraud-detection-data"
ref = "main"  # или другой commit/branch/tag
path = "ieee-fraud-detection/sample_submission.csv"

df = load_dataframe(configuration, repository, ref, path)
df.head()

In [ ]:
!pip install lakefs

In [ ]:
import lakefs
from lakefs.client import Client
import pandas as pd

clt = Client(
    host=os.getenv("LAKEFS_HOST"),
    username=os.getenv("LAKEFS_ACCESS_KEY"),
    password=os.getenv("LAKEFS_SECRET_KEY"),
)

# Укажите имя репозитория и ветки
repo_name = "mfdp-fin-fraud-detection-data"
branch_name = "main"
file_path = "ieee-fraud-detection/sample_submission.csv"

# Получите объект файла
obj = lakefs.repository(repo_name, client=clt).branch(branch_name).object(path=file_path)

# Прочитайте содержимое файла
with obj.reader(mode='r') as file:
    df = pd.read_csv(file)

df.head()


In [1]:
import pandas as pd

# Read into pandas directly by supplying the lakeFS URI...
sample_submission = pd.read_csv(f"lakefs://mfdp-fin-fraud-detection-data/main/ieee-fraud-detection/sample_submission.csv")
sample_submission.head()

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5


In [2]:
# Read into pandas directly by supplying the lakeFS URI...
train_identity = pd.read_csv(f"lakefs://mfdp-fin-fraud-detection-data/main/ieee-fraud-detection/train_identity.csv")
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [3]:
# Read into pandas directly by supplying the lakeFS URI...
train_transaction = pd.read_csv(f"lakefs://mfdp-fin-fraud-detection-data/main/ieee-fraud-detection/train_transaction.csv")
train_transaction.head()

PermissionException: code: 401, reason: Unauthorized, body: {}